[View in Colaboratory](https://colab.research.google.com/github/Leocodefocus/project/blob/master/multi_class.ipynb)

In [0]:
import tensorflow as tf
import pandas as pd

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [0]:
def get_data(y_name="Species"):
  train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
  test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)
  
  train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
  train_x, train_y = train, train.pop(y_name)

  test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
  test_x, test_y = test, test.pop(y_name)
  
  return (train_x,train_y),(test_x,test_y)


In [0]:
def train_input_fn(features,labels,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
  return dataset.shuffle(1000).repeat().batch(batch_size)

In [0]:
def eval_input_fn(features,labels,batch_size):
  features = dict(features)
  if labels is None:
    inputs = features
  else:
    inputs = (features,labels)
  dataset = tf.data.Dataset.from_tensor_slices(inputs)
  assert batch_size is not None,"Batch_size must not be None."
  dataset = dataset.batch(batch_size)
  return dataset

In [0]:
def construct_feature_columns(train_examples):
  my_feature_columns = []
  for key in train_examples.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
  return my_feature_columns

In [0]:
def train_model(train_examples,train_targets,test_examples,test_targets,
                batch_size,train_steps):
  my_feature_columns = construct_feature_columns(train_examples)
  classifier = tf.estimator.DNNClassifier(
      feature_columns = my_feature_columns,
      hidden_units = [10,10],
      n_classes=3)
  classifier.train(
      input_fn=lambda:train_input_fn(train_examples,train_targets,batch_size),
      steps = train_steps)
  eval_result = classifier.evaluate(
      input_fn=lambda:eval_input_fn(test_examples,test_targets,batch_size))
  print("\nTest set accuracy: {accuracy:0.3f}\n".format(**eval_result))
  expected = SPECIES
  predict_examples = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }
  predictions = classifier.predict(
      input_fn=lambda:eval_input_fn(predict_examples,
                                    labels = None,
                                    batch_size = batch_size))
  template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
  
  for pred_dict,expec in zip(predictions,expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print(template.format(expected[class_id],100*probability,expec))

In [16]:
tf.logging.set_verbosity(tf.logging.INFO)
(train_examples,train_targets),(test_examples,test_targets) = get_data()
train_model(train_examples,train_targets,test_examples,test_targets,
            batch_size=100,train_steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp73oveqcu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2e0995f908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp73oveqcu/model.ckpt.
INFO:tensorflow:loss = 164.80676, step = 1
INFO:tensorf